<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan; plan("avancé", "décorateur")

# les décorateurs


*avertissement* : version beta

# exemples

les deux constructions que nous avonc déjà rencontré

* `staticmethod`
* `classmethod`

sont des décorateurs

In [ ]:
# plutôt que d'écrire ceci

class C:

    @classmethod
    def f(c):
        pass

    @staticmethod
    def g():
        pass


In [ ]:
# on aurait aussi bien pu écrire ceci
# mais en moins lisible

class C:

    def f(c):
        pass

    def g():
        pass

    f = classmethod(f)
    g = staticmethod(g)

Le fragment

```
@decorateur
def f():
    pass
```

est en fait équivalent à

```
def f():
    pass
f = decorateur(f)
```

autrement dit, `f` n'est plus la fonction initiale, mais l'objet retourné par `decorateur(f)`

# qu'est-ce qu'un décorateur ?

c'est un *callable* qui prend en argument un *callable* et retourne un *callable*

# qu'est-ce qu'un *callable*

* le terme fait référence à un des nombreux protocoles
  * comme itérable qui veut dire 'peut être dans un for`
* ici un *callable* c'est, littéralement, un objet qu'on peut appeler
  * par exemple une fonction (bien entendu)
  * mais aussi une instance d'une classe qui implémente `__call__`

# une classe de *callables*

In [ ]:
# une classe dont les instances sont des callables
class Additioneur:

    # on crée une instance en lui passant la valeur à additionner
    def __init__(self, delta):
        self.delta = delta

    # ce qu'il faut faire à l'appel
    def __call__(self, entree):
        return entree + self.delta

In [ ]:
# ceci crée un callable
ajouter4 = Additioneur(4)
# qu'on peut donc utiliser comme une fonction
# en l'occurrence une fonction qui ajoute 4
ajouter4(10)

# à quoi sert un décorateur ?

à ajouter une couche de logique à une fonction avec une syntaxe explicite `@decorateur`

# comment implémenter un décorateur

* un décorateur est donc un *callable* (qui instrumente un *callable*)
* on peut donc choisir d'implémenter le décorateur comme
  * une fonction
  * une classe (avec le protocole `__call__`)

# exemple de décorateur - comme une classe

* pour qu'une fonction sache compter combien de fois elle est appelée
* le décorateur lui-même implémenté comme une classe

In [ ]:
class NumberCalls:

    # on aura une instance de NumberCalls
    # pour chaque fonction décorée
    # ceci est appelé à la déclaration de f
    def __init__(self, f):
        self.calls = 0
        self.f = f

    # et ce code est exécuté lors des appels à f
    def __call__(self, *args):
        self.calls += 1
        s = f'{self.f.__name__} : {self.calls} calls'
        print(s)
        return self.f(*args)

In [ ]:
# maintenant je peux définir une fonction décorée
@NumberCalls
def f(a, b):
    print(f"dans l'appel à f({a}, {b})")

In [ ]:
f(1, 2)

In [ ]:
f(3, 4)



# l'exemple décortiqué

la **déclaration** de `f`
```python
@NumberCalls
def f(a, b):
    print(blabla)
```

* devient
  * `f = NumberCalls(f)`
* qui déclenche
  * le **constructeur** de `NumberCalls`
  * avec `f` non décoré comme arg
  

* `f` décoré est une instance de `NumberCalls`
* qui est callable via `__call__`
* un **appel** à `f` décoré
```
f(1, 2)
```

* provoque maintenant 
  * un appel à **`__call__`** sur `f`
  * et avec arguments `(1, 2)`

# exemple - suite

* ce décorateur - implémenté comme une classe
* fonctionne bien sur des fonctions
* mais ça se passe moins bien avec des méthodes de classe

In [ ]:
class C:
    @NumberCalls
    def ma_methode(self, x):
        self.x = x

In [ ]:
c = C()
try:
    c.ma_methode(10)
except TypeError as e:
    print("OOPS", e)

* lors de l'appel à `c.ma_methode(10)`
* on appelle la méthode `__call__` sur l'instance de `NumberCalls`
* mais elle reçoit comme premier argument l'instance de `NumberCalls` (et non pas l'instance de `C`)
* et comme arguments dans `*args` uniquement l'entier `10`
* du coup `ma_methode` est appelée avec un seul argument `10` par `self.f(*args)`


# exemple de décorateur - comme une fonction

* même fonctionnalité 
* mais cette fois implémenté comme une fonction

In [ ]:
def NumberCalls2(f):

    ### le code exécuté à l'appel de f
    def wrapper(*args, **dargs):
        # on range le nombre d'appels directement
        # dans un attribut 'called' de l'objet fonction
        wrapper.calls += 1
        print(f'calling function {f.__name__}, '
              f'called {wrapper.calls} times')
        return f(*args, **dargs)

    ### le code exécuté à la déclaration de f
    # il faut initialiser cet attribut
    wrapper.calls = 0
    return wrapper

# exemple de décorateur - comme une fonction

In [ ]:
class D:
    @NumberCalls2
    def ma_methode(self, x):
        self.x = x

In [ ]:
d = D()
d.ma_methode(10)

# exemples de décorateurs

In [ ]:
from functools import wraps

def runtime(func):
    """
    Décorateur qui affiche le temps d'exécution d'une fonction
    """
    import time
    @wraps(func)
    def wrapper(*args, **kwargs):
        t = time.perf_counter()
        res = func(*args, **kwargs)
        print(func.__name__, time.perf_counter()-t)
        return res
    return wrapper


In [ ]:
def counter(func):
    """
    Décorateur qui affiche le nombre d'appels à une fonction 
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count = wrapper.count + 1
        res = func(*args, **kwargs)
        print("{} was called {} times".format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper


In [ ]:
def logfunc(func):
    """
    Décorateur qui log l'activité d'une fonction.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        s = """
The function *{}* was called with
    positional arguments: {}
    named arguments: {}
The returned value: {}
"""
        print(s.format(func.__name__, args, kwargs, res))
        return res
    return wrapper


In [ ]:
@logfunc
@counter
@runtime
def test(num, L):
    for i in range(num):
        'x' in L
    return 'Done'

test(100000, range(10))



# garder les métadonnées de la fonction décorée

In [ ]:
def mon_decorateur(func):
    def wrapper(*args, **kargs):
        print('avant func')
        func(*args, **kargs)
        print('apres func')
    return wrapper

@mon_decorateur
def ma_fonction(a, b):
    'une fonction qui ne fait presque rien'
    print('dans ma Fonction')
    print(a, b)

### garder les métadonnées de la fonction décorée

In [ ]:
ma_fonction(1, 2)

In [ ]:
print(ma_fonction.__doc__)

In [ ]:
print(ma_fonction.__name__)

### garder les métadonnées de la fonction décorée

* pour garder les métadonnées 
  * principalement les attributes `__doc__` et `__name_`
  * on .. décore le wrapper avec `functools.wraps`

### garder les métadonnées de la fonction décorée

In [ ]:
from functools import wraps

def mon_decorateur(func):
    @wraps(func)
    def wrapper(*args, **kargs):
        print('avant func')
        func(*args, **kargs)
        print('apres func')
    return wrapper

@mon_decorateur
def ma_fonction(a, b):
    'une fonction qui ne fait presque rien'
    print('dans ma Fonction')
    print(a, b)

### garder les métadonnées de la fonction décorée

In [ ]:
print(ma_fonction.__doc__)

In [ ]:
ma_fonction.__name__

In [ ]:
ma_fonction(1, 2)

In [ ]:
help(ma_fonction)

# cascader les décorateurs

In [ ]:
@runtime
@counter 
def f():
    pass

In [ ]:
# Est équivalent à 
f = runtime(counter(f))

### passer des arguments au décorateur

* on peut passer des argument au **décorateur**
  * ajouter une couche de logique
* en général, on utilise une fonction au dessus du décorateur 
  * dont le seul rôle est de permettre  
    au décorateur (fonction ou classe)

  * de garder un accès aux arguments par une clôture

In [ ]:
def nb_appel(label=''):
    class NumberCalls:
        def __init__(self, f):
            self.calls = 0
            self.f = f
        def __call__(self, *args):
            self.calls += 1
            s = (f'{label} {self.f.__name__} '
                 f': {self.calls} calls')
            print(s)
            return self.f(*args)
    return NumberCalls

In [ ]:
@nb_appel("-->")
def f(a, b):
    print(a, b)

f(1, 2)

In [ ]:
def caller_builder(label=''):
    def caller(f):
        def wrapper(*args, **dargs):
            wrapper.calls += 1
            print(f'{label} {f.__name__}, '
                  f'called {wrapper.calls} times')
            return f(*args, **dargs)
        wrapper.calls = 0
        return wrapper
    return caller

In [ ]:
class C:
    @caller_builder('method')
    def ma_methode(self, x):
        self.x = x

@caller_builder('function')
def ma_fonction():
    pass

In [ ]:
C().ma_methode(1)

In [ ]:
C().ma_methode(1)

In [ ]:
ma_fonction()

In [ ]:
ma_fonction()